In [1]:
import os
import pandas as pd

In [2]:
MOVEMENTS = (
    "elbflex",
    "shoabd",
    "shoext",
    "shoflex",
    "hipabd",
    "hipext",
    "hipflex",
    "kneeflex",
)

In [3]:
# Get path to participant & movement model generated CSVs.
# 
# trial_info dictionary:
# {
#   "dl200": {
#              "left_upper_elbflex": {"total_frames": 346, "good_frames": 70, "drop_percent": 0.80},
#              "left_upper_shoabd":  {...},
#              ...
#            },
#   "dl201": { ... },
#   ...
# }

trial_info = dict()
for movement in MOVEMENTS:
    path = f"./gen/{movement}/model/"
    for filename in os.listdir(path):
        if filename.endswith('.csv'):
            participant = filename.split('_')[0] # e.g. dl200
            group_and_movement = '_'.join(filename.split('_')[1:4]) # e.g. right_upper_elbflex
            trial_num = filename.strip('.csv').split('_')[-1]
            
            trial_df = pd.read_csv(f"{path}/{filename}", index_col=0)
            joint_name = [col for col in trial_df.columns if movement[:3] in col][0]
            
            total_frames = trial_df.shape[0]
            good_frames = trial_df[joint_name].dropna().shape[0]
            drop_percent = round((total_frames - good_frames) / total_frames, 2)
            
            trial_info.setdefault(participant, {})
            trial_info[participant].setdefault(group_and_movement, {})
            
            trial_info[participant][group_and_movement][trial_num] = {
                "total_frames": total_frames,
                "good_frames": good_frames,
                "drop_percent": drop_percent
            }

In [4]:
trial_info

{'dl200': {'right_upper_elbflex': {'024': {'total_frames': 346,
    'good_frames': 70,
    'drop_percent': 0.8},
   '027': {'total_frames': 341, 'good_frames': 66, 'drop_percent': 0.81},
   '007': {'total_frames': 343, 'good_frames': 117, 'drop_percent': 0.66},
   '026': {'total_frames': 346, 'good_frames': 73, 'drop_percent': 0.79},
   '029': {'total_frames': 325, 'good_frames': 67, 'drop_percent': 0.79},
   '031': {'total_frames': 319, 'good_frames': 112, 'drop_percent': 0.65},
   '005': {'total_frames': 332, 'good_frames': 113, 'drop_percent': 0.66},
   '023': {'total_frames': 314, 'good_frames': 79, 'drop_percent': 0.75},
   '006': {'total_frames': 320, 'good_frames': 112, 'drop_percent': 0.65},
   '009': {'total_frames': 325, 'good_frames': 109, 'drop_percent': 0.66},
   '021': {'total_frames': 295, 'good_frames': 69, 'drop_percent': 0.77},
   '010': {'total_frames': 323, 'good_frames': 83, 'drop_percent': 0.74},
   '022': {'total_frames': 305, 'good_frames': 68, 'drop_percent': 0

In [5]:
summary_dict = {}
for participant, gm_dict in trial_info.items():
    for group_and_movement, trial_dict in gm_dict.items():
        drop_percent_avg = 0
        for trial_num, info_dict in trial_dict.items():
            drop_percent_avg += info_dict['drop_percent']
        drop_percent_avg /= len(trial_dict)
        summary_dict.setdefault(participant, {})
        summary_dict[participant].setdefault(group_and_movement, {})
        summary_dict[participant][group_and_movement] = round(drop_percent_avg, 2)

In [6]:
summary_df = pd.DataFrame(summary_dict)
summary_df = summary_df.reindex(sorted(summary_df.columns), axis=1)
summary_df

,dl200,dl201,dl202,dl203,dl204,dl205
right_upper_elbflex,0.70,0.21,0.38,0.22,0.27,0.00
left_upper_elbflex,0.34,0.22,0.26,0.70,0.26,0.02
right_upper_shoabd,0.44,0.30,0.09,0.22,0.30,0.28
left_upper_shoabd,0.30,0.28,0.25,0.71,0.07,0.27
right_upper_shoext,0.79,0.31,0.51,0.24,0.02,0.01
left_upper_shoext,0.32,0.16,0.64,0.31,0.33,0.03
left_upper_shoflex,0.38,0.43,0.57,0.57,0.75,0.03
right_upper_shoflex,0.78,0.35,0.58,0.10,0.13,0.04
